In [2]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies.b_testing_strats import RSICross, SmaCross
from src.utils import load_data

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
ticker = "MSTR"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [4]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1998-06-11 00:00:00-04:00,8.00,12.25,7.94,10.56,10094800,0.00,0.00
1998-06-12 00:00:00-04:00,11.69,12.19,10.00,10.44,3321800,0.00,0.00
1998-06-15 00:00:00-04:00,10.25,10.75,9.38,10.22,1148000,0.00,0.00
1998-06-16 00:00:00-04:00,10.25,10.69,10.06,10.38,336400,0.00,0.00
1998-06-17 00:00:00-04:00,10.56,11.53,10.38,11.44,729000,0.00,0.00
...,...,...,...,...,...,...,...
2024-11-11 00:00:00-05:00,297.00,351.73,293.30,340.00,47315700,0.00,0.00
2024-11-12 00:00:00-05:00,334.77,360.89,325.50,356.59,39705800,0.00,0.00
2024-11-13 00:00:00-05:00,366.98,383.40,321.70,328.38,46238500,0.00,0.00


In [5]:
strategy = SmaCross
strategy.n1 = 50
strategy.n2 = 125
bt_sma = Backtest(data["2023":], strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     2023-01-03 00:00...
End                       2024-11-15 00:00...
Duration                    682 days 00:00:00
Exposure Time [%]                       48.94
Equity Final [$]                    53,194.21
Equity Peak [$]                     57,879.24
Return [%]                             431.94
Buy & Hold Return [%]                2,248.99
Return (Ann.) [%]                      144.08
Volatility (Ann.) [%]                  203.96
Sharpe Ratio                             0.71
Sortino Ratio                            3.51
Calmar Ratio                             3.10
Max. Drawdown [%]                      -46.41
Avg. Drawdown [%]                      -12.86
Max. Drawdown Duration      230 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                    2
Win Rate [%]                           100.00
Best Trade [%]                         229.37
Worst Trade [%]                         61.83
Avg. Trade [%]                    

In [6]:
bt_sma.plot()

Row(id='1450', ...)

In [7]:
%%time
stats = bt_sma.optimize(
    n1=range(20, 70, 10),
    n2=range(100, 250, 20),
    maximize='Equity Final [$]',
    constraint=lambda param: param.n1 < param.n2,
)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

CPU times: user 1.29 s, sys: 61 ms, total: 1.36 s
Wall time: 1.35 s


Start                     2023-01-03 00:00...
End                       2024-11-15 00:00...
Duration                    682 days 00:00:00
Exposure Time [%]                       50.00
Equity Final [$]                    64,642.89
Equity Peak [$]                     70,340.13
Return [%]                             546.43
Buy & Hold Return [%]                2,248.99
Return (Ann.) [%]                      170.85
Volatility (Ann.) [%]                  228.16
Sharpe Ratio                             0.75
Sortino Ratio                            4.17
Calmar Ratio                             3.68
Max. Drawdown [%]                      -46.39
Avg. Drawdown [%]                      -12.10
Max. Drawdown Duration      215 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                    2
Win Rate [%]                           100.00
Best Trade [%]                         227.36
Worst Trade [%]                         98.23
Avg. Trade [%]                    

In [7]:
bt_sma.plot()

Row(id='2182', ...)

In [8]:
strategy = RSICross
strategy.low_threshold = 36
strategy.high_threshold = 80
bt_rsi = Backtest(data["2023":], strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_rsi.run()
stats

Start                     2023-01-03 00:00...
End                       2024-11-15 00:00...
Duration                    682 days 00:00:00
Exposure Time [%]                       66.74
Equity Final [$]                   215,582.65
Equity Peak [$]                    215,582.65
Return [%]                           2,055.83
Buy & Hold Return [%]                2,248.99
Return (Ann.) [%]                      415.25
Volatility (Ann.) [%]                  408.78
Sharpe Ratio                             1.02
Sortino Ratio                           11.81
Calmar Ratio                            11.38
Max. Drawdown [%]                      -36.48
Avg. Drawdown [%]                       -8.50
Max. Drawdown Duration       78 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   17
Win Rate [%]                            41.18
Best Trade [%]                         196.28
Worst Trade [%]                         -9.29
Avg. Trade [%]                    

In [9]:
bt_rsi.plot()

Row(id='2207', ...)

In [10]:
%%time
stats = bt_rsi.optimize(
    low_threshold=range(20, 41, 2),
    high_threshold=range(60, 81, 2),
    maximize='Equity Final [$]',
)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 3.76 s, sys: 65.7 ms, total: 3.82 s
Wall time: 3.79 s


Start                     2023-01-03 00:00...
End                       2024-11-15 00:00...
Duration                    682 days 00:00:00
Exposure Time [%]                       66.74
Equity Final [$]                   215,582.65
Equity Peak [$]                    215,582.65
Return [%]                           2,055.83
Buy & Hold Return [%]                2,248.99
Return (Ann.) [%]                      415.25
Volatility (Ann.) [%]                  408.78
Sharpe Ratio                             1.02
Sortino Ratio                           11.81
Calmar Ratio                            11.38
Max. Drawdown [%]                      -36.48
Avg. Drawdown [%]                       -8.50
Max. Drawdown Duration       78 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   17
Win Rate [%]                            41.18
Best Trade [%]                         196.28
Worst Trade [%]                         -9.29
Avg. Trade [%]                    

In [11]:
stats._strategy

<Strategy RSICross(low_threshold=36,high_threshold=80)>